# Aufgabe 4:

In [1]:
#Import stuff & things
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

import pickle
import re
import nltk 

# Support-Functions

In [2]:
#Convenient function to output test items
def show_test_item(clf, i, text=True, classify=True):
    a = 'actual: ' + str(test.iloc[i].recommended)
    p = ''
    if classify:
        p = ' / predicted: ' + str(clf.predict(vectorizer.transform([test.iloc[i].content_text]))[0])
    print('[' + str(i) + '] ' +a + p)
    if text:
        print('\n')
        print(test.iloc[i].content_text)

In [3]:
def raw_tagger(tagger, text):
    sentences = sent_detector.tokenize(text)
    return [tagger.tag(re.findall('(\w+)', sentence)) for sentence in sentences]

   # Input Tagger

In [4]:
path = '/Users/juliusstahnke/Documents/HTW/Analytics/Analytics/Aufgabe4/'
filename = 'classifier-based-german-pos-tagger.pickle'
with open(path + filename, 'rb') as f:
    tagger = pickle.load(f)

In [5]:
sent_detector = nltk.data.load('tokenizers/punkt/german.pickle')

# Read Files

In [6]:
#Read Datasets for all Areas
dir = '/Users/juliusstahnke/Documents/HTW/Analytics/Analytics/Aufgabe4'
games = pd.read_json(dir + '/games/games.json', orient='records', encoding='utf8')
music = pd.read_json(dir + '/music/music.json', orient='records', encoding='utf8')
apps = pd.read_json(dir + '/apps/apps.json', orient='records', encoding='utf8')

# Split in Test and Train

In [7]:
#Set Randomn State
random_state = 100

#Set Traindata for games
train_games_positiv = games[games.recommended==True].sample(frac=0.8, random_state=random_state)
train_games_negativ = games[games.recommended==False].sample(frac=0.8, random_state=random_state)
train_games = pd.concat([train_games_positiv, train_games_negativ])
test_games = games.drop(train_games.index)

#Set Traindata for music
train_music_positiv = music[music.rating.isin(['8', '9', '10', '11', '12', '13', '14', '15'])].sample(frac=0.8, random_state=random_state)
train_music_negativ = music[music.rating.isin(['1', '2', '3', '4', '5', '6', '7'])].sample(frac=0.8, random_state=random_state)
train_music = pd.concat([train_music_positiv, train_music_negativ])
test_music = music.drop(train_music.index)

#Set Traindata for apps
train_apps_positiv = apps[apps.Rating >= 3].sample(frac=0.8, random_state=random_state)
train_apps_negativ = apps[apps.Rating < 3].sample(frac=0.8, random_state=random_state)
train_apps = pd.concat([train_apps_positiv, train_apps_negativ])
test_apps = apps.drop(train_apps.index)

# Normalize Tableformat

In [8]:
#Einfügen positiv Spalte und Text
#games.reset_index(inplace=True)
train_games['Positiv'] = (games.recommended)
train_games['Text'] = (games.content_text)

#Einfügen positiv Spalte und Text
#games.reset_index(inplace=True)
train_games['Positiv'] = (games.recommended)
train_games['Text'] = (games.content_text)

#Einfügen positiv Spalte und Text
train_apps = apps[apps.Rating!=3]
train_apps.reset_index(inplace=True)
train_apps['Positiv'] = (apps.Rating > 3)

#Einfügen positiv Spalte und Text
test_apps = apps[apps.Rating!=3]
test_apps.reset_index(inplace=True)
test_apps['Positiv'] = (apps.Rating > 3)

#Einfügen positiv Spalte und Text
train_music['Text'] = (music.description + music.summary)
train_music.reset_index(inplace=True)
train_music['Positiv'] = (music.rating.isin(['8', '9', '10', '11', '12', '13', '14', '15']))

#Einfügen positiver Spalten und Texte
test_music['Text'] = (music.description + music.summary)
test_music.reset_index(inplace=True)
test_music['Positiv'] = (music.rating.isin(['8', '9', '10', '11', '12', '13', '14', '15']))

/Users/juliusstahnke/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/juliusstahnke/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Vectorizer Games untagged

In [10]:
#Vectorizer Games
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(train_games.content_text)

X = vectorizer.transform(train_games.content_text)
y = train_games.recommended
clf = MultinomialNB().fit(X, y)
actuals = test_games.recommended
predicted = clf.predict(vectorizer.transform(test_games.content_text))
accuracy_score(actuals, predicted)

0.75987306064880111

# Tag Games

In [25]:
#train_games_tagged = [raw_tagger(tagger, a) for a in train_games.content_text]

#train_games_tagged

#PICKLE THE RESULT OF THIS STEP! THIS RUNS FOREVER!
#Here is how you pickle something:
#with open('games_pos_tagged.pickle', 'wb') as fp:
#    pickle.dump(train_games_tagged, fp)
    
#If you want to load your pickled stuff use the following: 
with open('games_pos_tagged.pickle', 'rb') as fp: 
    train_games_tagged = pickle.load(fp)

# Filter for Adjectives

In [26]:
#Declare result variabe
result_games_adja = []

#For every document (Needed because Reviews Contain more than one Sentence)
for document in train_games_tagged: 
    result_doc = ""
    #Check in Every Sentence (Needed Because Senteces have more than one word)
    for sentence in document:
        result_sen = ""
        #Check if the Second Element of the Wordtupel fullfills Condition
        for word in sentence:
            if word[1] == 'ADJA':
                #If so add it to Sentence, else dont
                result_sen = result_sen + " " + word[0]
        #If Sentence contains words that fullfill Condition add it to document, else dont 
        if len(result_sen) != 0:        
            result_doc = result_doc + result_sen
    #Add Document to Result (No Check if emty because we want reprensentation for every review in Trainingsdata)
    result_games_adja.append(result_doc)

## Vectorize for filtert adjectives

In [22]:
#Vectorizer Games
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(result_games_adja)

X = vectorizer.transform(result_games_adja)
y = train_games.recommended
clf = MultinomialNB().fit(X, y)
actuals = test_games.recommended
predicted = clf.predict(vectorizer.transform(test_games.content_text))
accuracy_score(actuals, predicted)

0.55835684062059243

# Filter for Adjectivs, adverbs, nouns and verbs

In [27]:
#Declare result variabe
result_games_multiple = []

#For every document (Needed because Reviews Contain more than one Sentence)
for document in train_games_tagged: 
    result_doc = ""
    #Check in Every Sentence (Needed Because Senteces have more than one word)
    for sentence in document:
        result_sen = ""
        #Check if the Second Element of the Wordtupel fullfills Condition
        for word in sentence:
            if word[1] == 'ADJA' or word[1] == 'ADV' or word[1] == 'NN' or 'VV' in word[1]:
                #If so add it to Sentence, else dont
                result_sen = result_sen + " " + word[0]
        #If Sentence contains words that fullfill Condition add it to document, else dont 
        if len(result_sen) != 0:        
            result_doc = result_doc + result_sen
    #Add Document to Result (No Check if emty because we want reprensentation for every review in Trainingsdata)
    result_games_multiple.append(result_doc)

# Vectorize for Adjectivs, adverbs, nouns and verbs

In [28]:
#Vectorizer Games
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(result_games_multiple)

X = vectorizer.transform(result_games_multiple)
y = train_games.recommended
clf = MultinomialNB().fit(X, y)
actuals = test_games.recommended
predicted = clf.predict(vectorizer.transform(test_games.content_text))
accuracy_score(actuals, predicted)

0.71738363892806767

# Vectorize Apps Untagged

In [29]:
vectorizer = CountVectorizer(min_df=105, max_df=200, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.79519799319118434

## Tag apps

In [9]:
train_apps_tagged = [raw_tagger(tagger, a) for a in train_apps.Text]

train_apps_tagged

#PICKLE THE RESULT OF THIS STEP! THIS RUNS FOREVER!
#Here is how you pickle something:
with open('apps_pos_tagged.pickle', 'wb') as fp:
    pickle.dump(train_apps_tagged, fp)
    
#If you want to load your pickled stuff use the following: 
#with open('games_pos_tagged.pickle', 'rb') as fp: 
#    test = pickle.load(fp)

# Filter for adjectivs

In [22]:
#Declare result variabe
result_apps_adja = []

#For every document (Needed because Reviews Contain more than one Sentence)
for document in train_apps_tagged: 
    result_doc = ""
    #Check in Every Sentence (Needed Because Senteces have more than one word)
    for sentence in document:
        result_sen = ""
        #Check if the Second Element of the Wordtupel fullfills Condition
        for word in sentence:
            if word[1] == 'ADJA':
                #If so add it to Sentence, else dont
                result_sen = result_sen + " " + word[0]
        #If Sentence contains words that fullfill Condition add it to document, else dont 
        if len(result_sen) != 0:        
            result_doc = result_doc + result_sen
    #Add Document to Result (No Check if emty because we want reprensentation for every review in Trainingsdata)
    result_apps_adja.append(result_doc)

# Vectorize for adjectives

In [14]:
#Vectorizer apps
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(result_apps_adja)

X = vectorizer.transform(result_apps_adja)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.79167413247327245

# Filter for Adjectivs, adverbs, nouns and verbs

In [21]:
#Declare result variabe
result_apps_multiple = []

#For every document (Needed because Reviews Contain more than one Sentence)
for document in train_apps_tagged: 
    result_doc = ""
    #Check in Every Sentence (Needed Because Senteces have more than one word)
    for sentence in document:
        result_sen = ""
        #Check if the Second Element of the Wordtupel fullfills Condition
        for word in sentence:
            if word[1] == 'ADJA' or word[1] == 'ADV' or word[1] == 'NN' or 'VV' in word[1]:
                #If so add it to Sentence, else dont
                result_sen = result_sen + " " + word[0]
        #If Sentence contains words that fullfill Condition add it to document, else dont 
        if len(result_sen) != 0:        
            result_doc = result_doc + result_sen
    #Add Document to Result (No Check if emty because we want reprensentation for every review in Trainingsdata)
    result_apps_multiple.append(result_doc)

# Vectorize for Adjectivs, adverbs, nouns and verbs

In [18]:
#Vectorizer Apps
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(result_apps_multiple)

X = vectorizer.transform(result_apps_multiple)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.79633279579525773

# Vectorize Music Untagged

In [19]:
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.64709705372616988

# Tag Music

In [20]:
train_music_tagged = [raw_tagger(tagger, a) for a in train_music.Text]

train_music_tagged

#PICKLE THE RESULT OF THIS STEP! THIS RUNS FOREVER!
#Here is how you pickle something:
with open('music_pos_tagged.pickle', 'wb') as fp:
    pickle.dump(train_music_tagged, fp)
    
#If you want to load your pickled stuff use the following: 
#with open('games_pos_tagged.pickle', 'rb') as fp: 
#    test = pickle.load(fp)

# Filter for Adjectiv


In [28]:
#Declare result variabe
result_music_adja = []

#For every document (Needed because Reviews Contain more than one Sentence)
for document in train_music_tagged: 
    result_doc = ""
    #Check in Every Sentence (Needed Because Senteces have more than one word)
    for sentence in document:
        result_sen = ""
        #Check if the Second Element of the Wordtupel fullfills Condition
        for word in sentence:
            if word[1] == 'ADJA':
                #If so add it to Sentence, else dont
                result_sen = result_sen + " " + word[0]
        #If Sentence contains words that fullfill Condition add it to document, else dont 
        if len(result_sen) != 0:        
            result_doc = result_doc + result_sen
    #Add Document to Result (No Check if emty because we want reprensentation for every review in Trainingsdata)
    result_music_adja.append(result_doc)

# Vectorize for Adjectiv

In [29]:
#Vectorizer music
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(result_music_adja)

X = vectorizer.transform(result_music_adja)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.62391681109185437

# Filter for Adjectivs, adverbs, nouns and verbs

In [34]:
#Declare result variabe
result_music_multiple = []

#For every document (Needed because Reviews Contain more than one Sentence)
for document in train_music_tagged: 
    result_doc = ""
    #Check in Every Sentence (Needed Because Senteces have more than one word)
    for sentence in document:
        result_sen = ""
        #Check if the Second Element of the Wordtupel fullfills Condition
        for word in sentence:
            if word[1] == 'ADJA' or word[1] == 'ADV' or word[1] == 'NN' or 'VV' in word[1]:
                #If so add it to Sentence, else dont
                result_sen = result_sen + " " + word[0]
        #If Sentence contains words that fullfill Condition add it to document, else dont 
        if len(result_sen) != 0:        
            result_doc = result_doc + result_sen
    #Add Document to Result (No Check if emty because we want reprensentation for every review in Trainingsdata)
    result_music_multiple.append(result_doc)

# Vectorize for Adjectivs, adverbs, nouns and verbs

In [35]:
#Vectorizer music
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(result_music_multiple)

X = vectorizer.transform(result_music_multiple)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.61243500866551126

# RegEx Hilfsfunktion

In [128]:
def addNegation(str):
    findNot = re.compile('(nicht.*?[,.!?:()])')
    findBegining = re.compile('(^|\s+)')

    str = re.split(findNot, str)
    i = 1

    while (i < len(str)-1):
        rstr = str[i][6:-1]
        newstr = re.sub(findBegining, " not_", rstr)
        str[i] = str[i].replace(" " + rstr, newstr)
        i = i+2
    
    result = ''
    for i in range(len(str)):
        result = result + str[i]


    return result

# Add Negation 

In [133]:
train_games_negated = [addNegation(str) for str in train_games.Text]    
test_games_negated = [addNegation(str) for str in test_games.content_text] 

train_apps_negated = [addNegation(str) for str in train_apps.Text]    
test_apps_negated = [addNegation(str) for str in test_apps.Text] 

train_music_negated = [addNegation(str) for str in train_music.Text]    
test_music_negated = [addNegation(str) for str in test_music.Text]

# Vectorize for Negated Games

In [131]:
#Vectorizer games
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(train_games_negated)

X = vectorizer.transform(train_games_negated)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.recommended
predicted = clf.predict(vectorizer.transform(test_games_negated))
accuracy_score(actuals, predicted)

0.76480959097320167

# Vectorize for Negated Apps

In [134]:
#Vectorizer apps
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(train_apps_negated)

X = vectorizer.transform(train_apps_negated)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps_negated))
accuracy_score(actuals, predicted)

0.80117063847578096

# Vectorize for Negated Music

In [135]:
#Vectorizer music
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(train_music_negated)

X = vectorizer.transform(train_music_negated)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music_negated))
accuracy_score(actuals, predicted)

0.64774696707105717